# Molecular Simulation 
Carrying out molecular dynamics (MD) and Monte-Carlo (MC) simulations using a (trained) potential. 

In [2]:
from pantea.simulation import (
    System,
    MDSimulator,
    BrendsenThermostat, 
    MCSimulator,
    simulate
)
from pantea.atoms import Structure
from pantea.units import units as units

import matplotlib.pylab as plt
from pathlib import Path
from ase import Atoms
from ase.visualize import view
import ase.io
import jax.numpy as jnp

In [3]:
# from pantea.logger import set_logging_level
# import logging
# set_logging_level(logging.DEBUG)

from pantea.types import default_dtype
default_dtype.FLOATX=jnp.float64

base_dir = Path('./LJ')

## Data

In [5]:
d = 6  # Angstrom
uc = Atoms('He', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
s0 = Structure.from_ase(uc.repeat((10, 10, 10)))

# d = 10  # Angstrom
# uc = Atoms('Ar', positions=[(d/2, d/2, d/2)], cell=(d, d, d))
# s0 = Structure.create_from_ase(uc.repeat((7, 7, 7)))

atoms = s0.to_ase()
# view(atoms, viewer='ngl') # ase, ngl

## Potential

In [6]:
from pantea.types import Array
import jax
from functools import partial
from pantea.simulation import LJPotential

# He
ljpot = LJPotential(
    sigma=2.5238 * units.FROM_ANGSTROM,  # Bohr
    epsilon=4.7093e-04 * units.FROM_ELECTRON_VOLT,  # Hartree
    r_cutoff=6.3095 * units.FROM_ANGSTROM,  # 2.5 * sigma
    # gradient_method="autodiff",
)

# Ar
# ljpot = LJPotential(
#     sigma=3.405 * units.FROM_ANGSTROM,                       # Bohr
#     epsilon=0.01032439284 * units.FROM_ELECTRON_VOLT,        # Hartree
#     r_cutoff=8.5125 * units.FROM_ANGSTROM,                   # 2.5 * sigma
# )

# ljpot(s0), ljpot.compute_forces(s0)

## Molecular Dynamics (MD)

In [7]:
time_step = 0.5 * units.FROM_FEMTO_SECOND
thermostat = BrendsenThermostat(target_temperature=300.0, time_constant=100 * time_step)
md = MDSimulator(time_step, thermostat)
sys = System.from_structure(s0, potential=ljpot, temperature=300.0)

In [8]:
sys.get_pressure()

Array(6.5147217e-07, dtype=float64)

In [9]:
sys.get_total_energy(), sys.get_potential_energy()

(Array(1.42327386, dtype=float64), Array(-0.00114392, dtype=float64))

In [10]:
# simulate(sys, md) # warm up
simulate(sys, md, num_steps=1000, output_freq=100) #, filename="md.xyz")

0          time[ps]:0.00000    Temp[K]:299.86371  Etot[Ha]:1.4232738554    Epot[Ha]:-0.0011439223   Pres[kb]:0.19167   
100        time[ps]:0.05000    Temp[K]:172.39433  Etot[Ha]:0.8178945368    Epot[Ha]:-0.0010159763   Pres[kb]:0.11015   
200        time[ps]:0.10000    Temp[K]:342.01576  Etot[Ha]:1.6237344159    Epot[Ha]:-0.0009147913   Pres[kb]:0.21862   
300        time[ps]:0.15000    Temp[K]:337.51732  Etot[Ha]:1.6024688583    Epot[Ha]:-0.0008117534   Pres[kb]:0.21574   
400        time[ps]:0.20000    Temp[K]:332.01006  Etot[Ha]:1.5763477416    Epot[Ha]:-0.0007721843   Pres[kb]:0.21218   
500        time[ps]:0.25000    Temp[K]:320.10393  Etot[Ha]:1.5198052249    Epot[Ha]:-0.0007580221   Pres[kb]:0.20457   
600        time[ps]:0.30000    Temp[K]:305.60832  Etot[Ha]:1.4509463936    Epot[Ha]:-0.0007595443   Pres[kb]:0.19532   
700        time[ps]:0.35000    Temp[K]:308.15700  Etot[Ha]:1.4630614870    Epot[Ha]:-0.0007512377   Pres[kb]:0.19697   
800        time[ps]:0.40000    Temp[K]:3

In [11]:
sys.get_pressure() * units.TO_KILO_BAR

Array(0.19149387, dtype=float64)

In [12]:
sys.positions * units.TO_ANGSTROM

Array([[9.50656466e-03, 4.41464602e+00, 4.97683978e+00],
       [5.70247836e+00, 2.26315646e+00, 1.25607508e+01],
       [8.75367435e-01, 4.07375875e+00, 1.61885311e+01],
       ...,
       [5.79100350e+01, 5.78075098e+01, 4.41341508e+01],
       [5.73856465e+01, 5.54044085e+01, 5.25861041e+01],
       [5.60884494e+01, 5.79520602e+01, 5.78570497e+01]], dtype=float64)

In [13]:
atoms = sys.structure.to_ase()
# view(atoms, viewer='ngl') # ase, ngl

## Monte Carlo (MC)

In [14]:
mc = MCSimulator(translate_step=0.3 * units.FROM_ANGSTROM, target_temperature=300, movements_per_step=10)
sys = System.from_structure(s0, potential=ljpot, temperature=300.0)

In [15]:
# simulate(sys, mc) # warm up
simulate(sys, mc, num_steps=1000, output_freq=100) #, filename=Path("mc.xyz"))

0          Epot[Ha]:-0.0011439223   
100        Epot[Ha]:-0.0011019419   
200        Epot[Ha]:-0.0010678284   
300        Epot[Ha]:-0.0010629610   
400        Epot[Ha]:-0.0010793298   
500        Epot[Ha]:-0.0011079703   
600        Epot[Ha]:-0.0011453637   
700        Epot[Ha]:-0.0011816754   
800        Epot[Ha]:-0.0012114927   
900        Epot[Ha]:-0.0012537843   
1000       Epot[Ha]:-0.0012923831   


In [16]:
sys.positions * units.TO_ANGSTROM

Array([[ 2.35166647,  3.93752599,  3.02452618],
       [ 2.21549591,  2.35946383,  8.38338638],
       [ 2.3917115 ,  2.67183769, 15.69858044],
       ...,
       [56.30426672, 57.14222665, 45.54597655],
       [56.61357868, 56.25949405, 51.17992683],
       [55.83441949, 57.01116821, 56.79909299]], dtype=float64)

In [17]:
atoms = sys.structure.to_ase()
# view(atoms, viewer='ngl') # ase, ngl